In [153]:
import numpy as np
import random
import copy

In [21]:
class Individual:
    
    def __init__(self, mentoriZauzetost, zeljeStudenata):
        self.mentoriZauzetost = copy.deepcopy(mentoriZauzetost)
        self.zeljeStudenata = copy.deepcopy(zeljeStudenata)
        self.brMentora = len(mentoriZauzetost)
        self.brStudenata = len(zeljeStudenata)
        self.maxBrojStudenata = sum(mentoriZauzetost)
        
        if self.maxBrojStudenata < self.brStudenata:
            print('Kapaciteti popunjeni! Nije moguce dodeliti mentora svakom studentu!')
        
        studenti = [i for i in range(0, self.brStudenata)]
        random.shuffle(studenti)
        mentori = []
        br=0
        for i in range(self.brMentora):
            for j in range(self.mentoriZauzetost[i]):
                if br + j > self.brStudenata:
                    break
                else :
                    mentori.append(i)
            br += j
        self.code = []
        for i in range(self.brStudenata):
            self.code.append((studenti[i], mentori[i]))
        self.fitness = self.fitnessFunction()
        
    def fitnessFunction(self):
        val = 0
        for i in range(self.brStudenata):
            val += self.geneFitness(i)
        return val
    
    def geneFitness(self, i):
        (student, mentor) = self.code[i]
        return self.zeljeStudenata[student][mentor]
    
    def __lt__(self, other):
        return self.fitness < other.fitness

In [22]:
def mutation(individual, mutation_rate):
    
    if random.uniform(0,1) < mutation_rate:
        i = random.randrange(individual.brStudenata)
        j = random.randrange(individual.brStudenata)
        tmp = individual.code[i][0]
        individual.code[i] = (individual.code[j][0], individual.code[i][1])
        individual.code[j] = (tmp, individual.code[j][1])
    

In [23]:
def selection(population, tournament_size):
    val = float('inf')
    bestInd = -1
    for _ in range(tournament_size):
        i = random.randrange(len(population))
        if population[i].fitness < val:
            val = population[i].fitness
            bestInd = i
    return bestInd

In [24]:
def crossover(parent1, parent2, child1, child2):
    
    n = parent1.brStudenata
    dodeljeni1 = set()
    dodeljeni2 = set()
    nisuDodeljenji1 = set()
    nisuDodeljenji2 = set()
    for i in range(n):
        nisuDodeljenji1.add(parent1.code[i][0])
        nisuDodeljenji2.add(parent1.code[i][0])
    for i in range(n):
        child1.code[i] = (-1, child1.code[i][1])
        child2.code[i] = (-1, child2.code[i][1])
    for i in range(n):
        if parent1.geneFitness(i) < parent2.geneFitness(i) and parent1.code[i][0] not in dodeljeni1 and parent1.code[i][0] in nisuDodeljenji1:
            child1.code[i] = (parent1.code[i][0], child1.code[i][1])
            dodeljeni1.add(child1.code[i][0])
            nisuDodeljenji1.remove(child1.code[i][0])
        elif parent1.geneFitness(i) > parent2.geneFitness(i) and parent2.code[i][0] not in dodeljeni1 and parent2.code[i][0] in nisuDodeljenji1:
            child1.code[i] = (parent2.code[i][0], child1.code[i][1])
            dodeljeni1.add(child1.code[i][0])
            nisuDodeljenji1.remove(child1.code[i][0])
    
    k=n-1
    while k>=0:
        if parent1.geneFitness(k) < parent2.geneFitness(k) and parent1.code[k][0] not in dodeljeni2 and parent1.code[k][0] in nisuDodeljenji2:
            child2.code[k] = (parent1.code[k][0], child2.code[k][1])
            dodeljeni2.add(child2.code[k][0])
            nisuDodeljenji2.remove(child2.code[k][0])
        elif parent1.geneFitness(k) > parent2.geneFitness(k) and parent2.code[k][0] not in dodeljeni2 and parent2.code[k][0] in nisuDodeljenji2:
            child2.code[k] = (parent2.code[k][0], child2.code[k][1])
            dodeljeni2.add(child2.code[k][0])
            nisuDodeljenji2.remove(child2.code[k][0])
        k-=1
    
    for i in range(n):
        if child1.code[i][0] == -1:
           
            child1.code[i] = (random.sample(nisuDodeljenji1, 1)[0], child1.code[i][1])
            
            nisuDodeljenji1.remove(child1.code[i][0])
        if child2.code[i][0] == -1:
           
            child2.code[i] = (random.sample(nisuDodeljenji2, 1)[0], child2.code[i][1])
            
            nisuDodeljenji2.remove(child2.code[i][0])
            
    child1.fitness = child1.fitnessFunction()
    child2.fitness = child2.fitnessFunction()
    
    if child1.fitness > parent1.fitness:
        child1 = parent1
    if child1.fitness > parent2.fitness:
        child1 = parent2
    if child2.fitness > parent1.fitness:
        child2 = parent1
    if child2.fitness > parent2.fitness:
        child2 = parent2

In [89]:
def generisiPrimer(brMentora, brStudenata):
    mentoriZauzetost = [random.randint(int(brStudenata/brMentora)-int(brStudenata/(brMentora*4)),int(brStudenata/brMentora)+2*int(brStudenata/(brMentora*4))) for i in range(brMentora)]
    zeljeStudenata = [[int(i) for i in range(brMentora)] for j in range(brStudenata)]

    for i in range(len(zeljeStudenata)):
        random.shuffle(zeljeStudenata[i])
    
    return mentoriZauzetost, zeljeStudenata
    

In [128]:
mentoriZauzetost, zeljeStudenata = generisiPrimer(15, 60)


60 15


In [145]:
# mentoriZauzetost, zeljeStudenata = generisiPrimer(10, 50)
population_size = 100
mutation_rate = 0.1
tournament_size = 6
max_iter = 1000
elitism_size = 20
population = [Individual(mentoriZauzetost, zeljeStudenata) for _ in range(population_size)]
new_population = [Individual(mentoriZauzetost, zeljeStudenata) for _ in range(population_size)]
# for i in range(population_size):
#     print(population[i].code)

for iteration in range(max_iter):
    population.sort()
    
    
    #print(iteration,'-ta iteracija, najbolji fitness je: ', population[0].fitness)
    if population[0].fitness == 0:
        print("Svakom studentu je ispunjena prva zelja!")
        break
    for i in range(elitism_size):
        new_population[i] = population[i]
    for i in range(elitism_size, population_size, 2):
        p1 = selection(population, tournament_size)
        p2 = selection(population, tournament_size)
        
        crossover(population[p1], population[p2], new_population[i], new_population[i+1])
        
        mutation(new_population[i], mutation_rate)
        mutation(new_population[i+1], mutation_rate)
        new_population[i].fitness = new_population[i].fitnessFunction()
        new_population[i+1].fitness = new_population[i+1].fitnessFunction()
        
    population = new_population

population.sort()



najbolje resenje je:
[(54, 0), (34, 0), (7, 0), (47, 1), (39, 1), (27, 1), (21, 1), (3, 1), (55, 2), (42, 2), (46, 2), (19, 2), (5, 3), (49, 3), (1, 3), (31, 3), (43, 3), (11, 3), (45, 4), (35, 4), (50, 4), (13, 4), (9, 5), (28, 5), (6, 5), (30, 5), (24, 5), (51, 5), (58, 6), (40, 6), (52, 6), (17, 6), (33, 6), (15, 7), (32, 7), (20, 7), (56, 7), (0, 7), (22, 8), (2, 8), (37, 8), (16, 8), (10, 8), (38, 8), (53, 9), (26, 9), (36, 9), (59, 9), (23, 9), (44, 10), (48, 10), (14, 10), (57, 10), (18, 10), (25, 11), (29, 11), (4, 11), (8, 11), (12, 11), (41, 12)]
54


In [148]:
brPrvihZelja = 0
najgoraZelja = 0
ispunjeneZelje = {}
for i in range(len(population[0].code)):
    print("{}. studentu je dodeljen {}. i ispunjena mu je {}. zelja".format(population[0].code[i][0], population[0].code[i][1], population[0].geneFitness(i)))
    if population[0].geneFitness(i) == 0:
        brPrvihZelja +=1
    if population[0].geneFitness(i) > najgoraZelja:
        najgoraZelja = population[0].geneFitness(i)
    if population[0].geneFitness(i) not in ispunjeneZelje:
        ispunjeneZelje[population[0].geneFitness(i)] = 1
    else:
        ispunjeneZelje[population[0].geneFitness(i)] +=1
print("{}% studenata je ispunjena prva zelja".format((brPrvihZelja*100)/population[0].brStudenata))
print("najgora ispunjena zelja se nalazi u prvih {}% zelja na listi".format((najgoraZelja*100)/population[0].brMentora))

print(ispunjeneZelje)

0. studentu je dodeljen 7. i ispunjena mu je 2. zelja
1. studentu je dodeljen 3. i ispunjena mu je 2. zelja
2. studentu je dodeljen 8. i ispunjena mu je 1. zelja
3. studentu je dodeljen 1. i ispunjena mu je 2. zelja
4. studentu je dodeljen 11. i ispunjena mu je 5. zelja
5. studentu je dodeljen 3. i ispunjena mu je 0. zelja
6. studentu je dodeljen 5. i ispunjena mu je 0. zelja
7. studentu je dodeljen 0. i ispunjena mu je 0. zelja
8. studentu je dodeljen 11. i ispunjena mu je 0. zelja
9. studentu je dodeljen 5. i ispunjena mu je 0. zelja
10. studentu je dodeljen 8. i ispunjena mu je 2. zelja
11. studentu je dodeljen 3. i ispunjena mu je 0. zelja
12. studentu je dodeljen 11. i ispunjena mu je 0. zelja
13. studentu je dodeljen 4. i ispunjena mu je 2. zelja
14. studentu je dodeljen 10. i ispunjena mu je 0. zelja
15. studentu je dodeljen 7. i ispunjena mu je 0. zelja
16. studentu je dodeljen 8. i ispunjena mu je 1. zelja
17. studentu je dodeljen 6. i ispunjena mu je 2. zelja
18. studentu je 

In [152]:

# brStudenata = len(zeljeStudenata)
# brMentora = len(mentoriZauzetost)
# mentoriKapacitet = sum(mentoriZauzetost)

In [150]:
# !pip install python-constraint

In [151]:
# import constraint
# mapirajMentore = {}
# br=0
# for i in range(len(mentoriZauzetost)):
#     for j in range(mentoriZauzetost[i]):
        
#         mapirajMentore[br] = i
#         br += 1

In [135]:
# problem = constraint.Problem()


In [136]:
# npr niz[i]=j oznacava da je mentoru mapirajMentore[i] dodeljen j-ti student
# problem.addVariables(['mentor_{}'.format(i) for i in range(min(mentoriKapacitet, brStudenata))], range(brStudenata))
# problem.addConstraint(constraint.AllDifferentConstraint())


In [137]:
# solutions = problem.getSolutions()

In [138]:
# min_val = brStudenata*brMentora
# najbolje_resenje = None

# for sol in range solutions:
#     val = 0
#     for example in sol:
    
#         ment = int(example.strip('mentor_'))
#         print('student {} = {}'.format(sol[example], zeljeStudenata[int(sol[example])][mapirajMentore[ment]]))
#         val += zeljeStudenata[int(sol[example])][mapirajMentore[ment]]
#     if min_val>val:
#         min_val = val
#         najbolje_resenje = sol


In [139]:
# print(val)